# T1034 Path Interception

-----------------------------------------------------------------------

## Technique Description

**This technique has been deprecated. Please use [Path Interception by PATH Environment Variable](https://attack.mitre.org/techniques/T1574/007), [Path Interception by Search Order Hijacking](https://attack.mitre.org/techniques/T1574/008), and/or [Path Interception by Unquoted Path](https://attack.mitre.org/techniques/T1574/009).**

Path interception occurs when an executable is placed in a specific path so that it is executed by an application instead of the intended target. One example of this was the use of a copy of [cmd](https://attack.mitre.org/software/S0106) in the current working directory of a vulnerable application that loads a CMD or BAT file with the CreateProcess function. (Citation: TechNet MS14-019)

There are multiple distinct weaknesses or misconfigurations that adversaries may take advantage of when performing path interception: unquoted paths, path environment variable misconfigurations, and search order hijacking. The first vulnerability deals with full program paths, while the second and third occur when program paths are not specified. These techniques can be used for persistence if executables are called on a regular basis, as well as privilege escalation if intercepted executables are started by a higher privileged process.

### Unquoted Paths
Service paths (stored in Windows Registry keys) (Citation: Microsoft Subkey) and shortcut paths are vulnerable to path interception if the path has one or more spaces and is not surrounded by quotation marks (e.g., <code>C:\unsafe path with space\program.exe</code> vs. <code>"C:\safe path with space\program.exe"</code>). (Citation: Baggett 2012) An adversary can place an executable in a higher level directory of the path, and Windows will resolve that executable instead of the intended executable. For example, if the path in a shortcut is <code>C:\program files\myapp.exe</code>, an adversary may create a program at <code>C:\program.exe</code> that will be run instead of the intended program. (Citation: SecurityBoulevard Unquoted Services APR 2018) (Citation: SploitSpren Windows Priv Jan 2018)

### PATH Environment Variable Misconfiguration
The PATH environment variable contains a list of directories. Certain methods of executing a program (namely using cmd.exe or the command-line) rely solely on the PATH environment variable to determine the locations that are searched for a program when the path for the program is not given. If any directories are listed in the PATH environment variable before the Windows directory, <code>%SystemRoot%\system32</code> (e.g., <code>C:\Windows\system32</code>), a program may be placed in the preceding directory that is named the same as a Windows program (such as cmd, PowerShell, or Python), which will be executed when that command is executed from a script or command-line.

For example, if <code>C:\example path</code> precedes <code>C:\Windows\system32</code> is in the PATH environment variable, a program that is named net.exe and placed in <code>C:\example path</code> will be called instead of the Windows system "net" when "net" is executed from the command-line.

### Search Order Hijacking
Search order hijacking occurs when an adversary abuses the order in which Windows searches for programs that are not given a path. The search order differs depending on the method that is used to execute the program. (Citation: Microsoft CreateProcess) (Citation: Hill NT Shell) (Citation: Microsoft WinExec) However, it is common for Windows to search in the directory of the initiating program before searching through the Windows system directory. An adversary who finds a program vulnerable to search order hijacking (i.e., a program that does not specify the path to an executable) may take advantage of this vulnerability by creating a program named after the improperly specified program and placing it within the initiating program's directory.

For example, "example.exe" runs "cmd.exe" with the command-line argument <code>net user</code>. An adversary may place a program called "net.exe" within the same directory as example.exe, "net.exe" will be run instead of the Windows system utility net. In addition, if an adversary places a program called "net.com" in the same directory as "net.exe", then <code>cmd.exe /C net user</code> will execute "net.com" instead of "net.exe" due to the order of executable extensions defined under PATHEXT. (Citation: MSDN Environment Property)

Search order hijacking is also a common practice for hijacking DLL loads and is covered in [DLL Search Order Hijacking](https://attack.mitre.org/techniques/T1038).

## Technique Detection

Monitor file creation for files named after partial directories and in locations that may be searched for common processes through the environment variable, or otherwise should not be user writable. Monitor the executing process for process executable paths that are named for partial directories. Monitor file creation for programs that are named after Windows system programs or programs commonly executed without a path (such as "findstr," "net," and "python"). If this activity occurs outside of known administration activity, upgrades, installations, or patches, then it may be suspicious. 

Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as network connections made for Command and Control, learning details about the environment through Discovery, and Lateral Movement.

-----------------------------------------------------------------------

### Tactics:

  * Persistence

  * Privilege-Escalation

### Platforms:

  * Windows

### Adversary Required Permissions:

  * User

  * Administrator

  * SYSTEM

### Effective Permissions:

  * User

  * Administrator

  * SYSTEM

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1034)

  * [Capec](https://capec.mitre.org/data/definitions/159.html)

  * [Technet Ms14-019](https://blogs.technet.microsoft.com/srd/2014/04/08/ms14-019-fixing-a-binary-hijacking-via-cmd-or-bat-file/), Nagaraju, S. (2014, April 8). MS14-019 – Fixing a binary hijacking via .cmd or .bat file. Retrieved July 25, 2016.

  * [Microsoft Subkey](http://support.microsoft.com/KB/103000), Microsoft. (n.d.). CurrentControlSet\Services Subkey Entries. Retrieved November 30, 2014.

  * [Baggett 2012](https://isc.sans.edu/diary/Help+eliminate+unquoted+path+vulnerabilities/14464), Baggett, M. (2012, November 8). Help eliminate unquoted path vulnerabilities. Retrieved December 4, 2014.

  * [Securityboulevard Unquoted Services Apr 2018](https://securityboulevard.com/2018/04/windows-privilege-escalation-unquoted-services/), HackHappy. (2018, April 23). Windows Privilege Escalation – Unquoted Services. Retrieved August 10, 2018.

  * [Sploitspren Windows Priv Jan 2018](https://www.sploitspren.com/2018-01-26-Windows-Privilege-Escalation-Guide/), McFarland, R. (2018, January 26). Windows Privilege Escalation Guide. Retrieved August 10, 2018.

  * [Microsoft Createprocess](http://msdn.microsoft.com/en-us/library/ms682425), Microsoft. (n.d.). CreateProcess function. Retrieved December 5, 2014.

  * [Hill Nt Shell](http://technet.microsoft.com/en-us/library/cc723564.aspx#XSLTsection127121120120), Hill, T. (n.d.). Windows NT Command Shell. Retrieved December 5, 2014.

  * [Microsoft Winexec](http://msdn.microsoft.com/en-us/library/ms687393), Microsoft. (n.d.). WinExec function. Retrieved December 5, 2014.

  * [Msdn Environment Property](https://msdn.microsoft.com/en-us/library/fd7hxfdd.aspx), Microsoft. (n.d.). Environment Property. Retrieved July 27, 2016.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------
### This technique is a duplicate.  Follow the link below to the "Primary Version".
<a href="../Persistence/T1034 Path Interception.ipynb" target="_blank">Primary Version</a>